In [2]:
import jax
import flax
import hard_not
import harden


In [7]:
soft_not, hard_not = hard_not.NotLayer(layer_size=4)
soft_weights = soft_not.init(jax.random.PRNGKey(0), [1.0, 1.0])
hard_weights = harden.harden_dict(soft_weights)
[
    soft_not.apply(soft_weights, [1.0, 1.0]),
    hard_not.apply(hard_weights, [True, True])
]

[DeviceArray([[0.26377404, 0.8707025 ],
              [0.44444847, 0.03216302],
              [0.6110164 , 0.685097  ],
              [0.9133855 , 0.08662593]], dtype=float32),
 DeviceArray([[False,  True],
              [False, False],
              [ True,  True],
              [ True, False]], dtype=bool)]

In [17]:
closed_soft_jax_expr = jax.make_jaxpr(soft_not.apply)(soft_weights, jax.numpy.array([1.0, 1.0]))
closed_hard_jax_expr = jax.make_jaxpr(hard_not.apply)(hard_weights, jax.numpy.array([True, True]))

In [18]:
print(closed_soft_jax_expr)

{ lambda ; a:f32[4,2] b:f32[2]. let
    c:f32[4,2] = xla_call[
      call_jaxpr={ lambda ; d:f32[4,2] e:f32[] f:f32[]. let
          g:f32[] = convert_element_type[new_dtype=float32 weak_type=False] e
          h:f32[4,2] = max g d
          i:f32[] = convert_element_type[new_dtype=float32 weak_type=False] f
          j:f32[4,2] = min i h
        in (j,) }
      name=clip
    ] a 0.0 1.0
    k:f32[4,2] = sub 1.0 c
    l:f32[4,2] = mul 2.0 c
    m:f32[4,2] = sub l 1.0
    n:f32[1,2] = broadcast_in_dim[broadcast_dimensions=(1,) shape=(1, 2)] b
    o:f32[4,2] = mul n m
    p:f32[4,2] = add k o
  in (p,) }


In [19]:
print(closed_hard_jax_expr)

{ lambda ; a:bool[4,2] b:bool[2]. let
    c:bool[4,2] = xla_call[
      call_jaxpr={ lambda ; d:bool[4,2] e:bool[2]. let
          f:bool[1,2] = broadcast_in_dim[broadcast_dimensions=(1,) shape=(1, 2)] e
          g:bool[4,2] = xor f d
          h:bool[4,2] = not g
        in (h,) }
      name=hard_not
    ] a b
  in (c,) }


In [ ]:
for eqn in closed_soft_jax_expr.eqns:
    print(eqn)